In [1]:
import os

In [2]:
%pwd

'c:\\Users\\LENOVO\\1. Projects\\Bodycode-Anomaly-Detection-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\LENOVO\\1. Projects\\Bodycode-Anomaly-Detection-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    scaler_path: Path
    metric_file_name: Path

In [6]:
from bodycoteAnomalyDetection.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from bodycoteAnomalyDetection.utils.common import read_yaml, create_directories

[2024-01-05 13:06:13,438: INFO: utils: NumExpr defaulting to 8 threads.]


In [7]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath=CONFIG_FILE_PATH, 
            params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            scaler_path = config.scaler_path,
            metric_file_name = config.metric_file_name
           
        )

        return model_evaluation_config

In [8]:
import pandas as pd
from keras.models import load_model
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
import json
import joblib
from bodycoteAnomalyDetection.components import model_trainer

[2024-01-05 13:06:27,524: WARNING: module_wrapper: From c:\Users\LENOVO\anaconda3\Lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]


In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        self.trainer = model_trainer.ModelTrainer(config)
        self.model = self.load_model()
        self.data = self.load_data()
        

    def load_model(self):
        return load_model(self.config.model_path)

    def load_data(self):
        df = self.trainer.load_data()
        scaler = joblib.load(self.config.scaler_path)
        X_scaled = scaler.transform(df)
        X_reshaped = self.trainer.data_reshaper(X_scaled)
        return X_reshaped

    def evaluate(self):
        predictions = self.model.predict(self.data)
        data_flat = self.trainer.data_flattening(self.data)
        predictions_flat = self.trainer.data_flattening(predictions)
        mse = mean_squared_error(data_flat, predictions_flat)
        mae = mean_absolute_error(data_flat, predictions_flat)
        r2 = r2_score(data_flat, predictions_flat)

        metrics = {
            "MSE": mse,
            "MAE": mae,
            "R2 Score": r2
        }
        with open(self.config.metric_file_name, 'w') as metric_file:
            json.dump(metrics, metric_file)

        return metrics

    def display_evaluation(self):
        """Display or log the evaluation results."""
        metrics = self.evaluate()
        print("Evaluation Metrics:")
        for metric, value in metrics.items():
            print(f"{metric}: {value}")

In [10]:
try:
    config_manager = ConfigurationManager()
    model_eval_config = config_manager.get_model_evaluation_config()
    model_evaluator = ModelEvaluation(config=model_eval_config)
    model_evaluator.evaluate()
except Exception as e:
    raise e

[2024-01-05 13:06:28,515: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-05 13:06:28,525: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-05 13:06:28,528: INFO: common: created directory at: artifacts]
[2024-01-05 13:06:28,530: INFO: common: created directory at: artifacts/model_evaluation]
[2024-01-05 13:06:28,948: WARNING: module_wrapper: From c:\Users\LENOVO\anaconda3\Lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.
]
4998/4998 [==============================] - 36s 7ms/step
Evaluation Metrics:
MSE: 0.0011828201164277108
MAE: 0.013798639101330702
R2 Score: 0.31597236901631337
